Students implement Hill-Climbing Search, Local Beam Search, and Simulated Annealing Search algorithms following TODO 1 - 3. \
Students can add supporting attributes and methods to the three classes as needed.

# Libraries

In [41]:
import os
import heapq

# Graph class

In [42]:
# Directed, weighted graphs
class Graph:
  def __init__(self):
    self.AL = dict() # adjacency list
    self.V = 0
    self.E = 0
    self.H = dict()

  def __str__(self):
    res = 'V: %d, E: %d\n'%(self.V, self.E)
    for u, neighbors in self.AL.items():
      line = '%d: %s\n'%(u, str(neighbors))
      res += line
    for u, h in self.H.items():
      line = 'h(%d) = %d\n'%(u, h)
    return res

  def print(self):
    print(str(self))

  def load_from_file(self, filename):
    '''
        Example input file:
            V E
            u v w
            u v w
            u v w
            ...
            u1 h1
            u2 h2
            u3 h3
            ...

        # input.txt
        7 8
        0 1 5
        0 2 6
        1 3 12
        1 4 9
        2 5 5
        3 5 8
        3 6 7
        4 6 4
        0 14
        1 13
        2 12
        3 11
        4 10
        5 9
        6 0
    '''
    if os.path.exists(filename):
      with open(filename) as g:
        self.V, self.E = [int(it) for it in g.readline().split()]
        for i in range(self.E):
          line = g.readline()
          u, v, w = [int(it) for it in line.strip().split()]
          if u not in self.AL:
            self.AL[u] = []
          self.AL[u].append((v, w))
        for i in range(self.V):
          line = g.readline()
          u, h = [int(it) for it in line.strip().split()]
          self.H[u] = h

In [43]:
g = Graph()
g.load_from_file('input.txt')
g.print()

V: 7, E: 8
0: [(1, 5), (2, 6)]
1: [(3, 12), (4, 9)]
2: [(5, 5)]
3: [(5, 8), (6, 7)]
4: [(6, 4)]



# Search Strategies

In [44]:
class LocalSearchStrategy:
  def search(self, g: Graph, src: int) -> tuple:
    '''
    return a tuple (u, p) in which
      u: the local maximum state
      p: the priority/weight/desirability/cost of u
    '''
    return (None, None)

In [45]:
class HillClimbingSearch(LocalSearchStrategy):
  def search(self, g: Graph, src: int) -> tuple:
    '''
    return a tuple (u, p) in which
      u: the local maximum state
      p: the priority/weight/desirability/cost of u

    Note: weight of a node u = path_cost to u + heuristic value of u (similar to A*)
    '''
    path_cost = {
      src: 0
    }
    
    current = (src, 0 + g.H[src])
    while(True):
      if current[0] not in g.AL:
        break
      
      neighbors = g.AL[current[0]]
      
      for neighbor in neighbors:
        path_cost[neighbor[0]] = path_cost.get(current[0],0) + neighbor[1]

      weights = [(node, path_cost[node] + g.H[node]) for node, weight in neighbors]
      max_node = max(weights, key=lambda x: x[1])
      
      if max_node[1] > current[1]:
        current = max_node
      else:
        break
     
    return current

In [46]:
import random 
class LocalBeamSearch(LocalSearchStrategy):
	def __init__(self) -> None:
		self.path_cost = dict()
	
	def search(self, g: Graph, src: int) -> tuple:
		'''
		return a tuple (u, p) in which
			u: the local maximum state
			p: the priority/weight/desirability/cost of u

		Note: weight of a node u = path_cost to u + heuristic value of u (similar to A*)
		'''
		# expand neighbors from src
		self.path_cost[src] = 0
		neighbors = g.AL[src]
		for neighbor in neighbors:
			self.path_cost[neighbor[0]] = self.path_cost.get(src, 0) + neighbor[1]

		# select k states randomly from src's neighbors
		current_nodes = [{ "state": (node, w), "score": self.path_cost[node] + g.H[node] } for node, w in neighbors]
		k = random.randint(1, len(current_nodes))	

		current_states = random.sample(current_nodes, k)
	
		result_node = (None, None)
		while True:
			# travel each node in current_states
			current_layer = []
			for node in current_nodes:
				if node["state"][0] not in g.AL:
					result_node = (node["state"][0], node["score"])
					continue
     
				neighbors = g.AL[node["state"][0]]
    
				for neighbor in neighbors:
					self.path_cost[neighbor[0]] = self.path_cost.get(node["state"][0], 0) + neighbor[1]
				weights = [{ "state": (node, w), "score": self.path_cost[node] + g.H[node]} for node, w in neighbors]
				max_node = max(weights, key=lambda x: x["score"])
				
				if max_node["score"] < node["score"]:
					return (node["state"][0], node["score"])

				current_layer = current_layer + weights

			if len(current_layer) == 0:
				break
			current_nodes = random.sample(current_layer, k)
		return result_node
  

In [47]:
import math
import random
class SimulatedAnnealingSearch(LocalSearchStrategy):
  def schedule(self, t: int) -> float:
    return 1/(t**2)
  
  def search(self, g: Graph, src: int) -> tuple:
    '''
    return a tuple (u, p) in which
      u: the local maximum state
      p: the priority/weight/desirability/cost of u

    Note: schedule(t) = 1/(t^2) with t is the iteration step
    '''
    path_cost = {
			src: 0
		}
    weight = {
			src: 0 + g.H[src]
		}
    current = (src, 0 + g.H[src])
    t = 1
    while True:
      T = self.schedule(t)
      if T == 0:
        return current
      if current[0] not in g.AL:
        break
      neighbors = g.AL[current[0]]
      for neighbor in neighbors:
        path_cost[neighbor[0]] = path_cost.get(current[0],0) + neighbor[1]
        weight[neighbor[0]] = path_cost[neighbor[0]] + g.H[neighbor[0]]
      next_node = random.choice(neighbors)
      delta_e = weight[next_node[0]] - weight[current[0]]
      if delta_e > 0:
        current = next_node
      else:
        current = next_node if random.random() < math.exp(delta_e/T) else current
    return (current[0], weight[current[0]])

# Evaluation

In [48]:
hcs = HillClimbingSearch()
lbs = LocalBeamSearch()
sas = SimulatedAnnealingSearch()

for stg in [hcs, lbs, sas]:
  print(stg)
  u, p = stg.search(g, 0)
  print(u, p)

5 34
4 24
6 18


# Submission

*   Students download the notebook after completion
*   Rename the notebook in which inserting your student ID at the beginning. \
For example, **123456-LocalSearch-HW.ipynb**
*   Finally, submit the file